# LibTetraBZ tutorial
This is tutorial for [LibtetraBZ](https://libtetrabz.osdn.jp/) which is a libraly to perform Brillouin-zone integrals with the optimized tetrahedron method [1].

[1] "Improved tetrahedron method for the Brillouin-zone integration applicable to response functions", M. Kawamura, Y. Gohda, and S. Tsuneyuki, [Phys. Rev. B 89, 094515](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.89.094515) (2014).

## 1, Preparation
Before we start this tutorial, the nessesarry packages should be installed.
LibTeteraBZ package is installed through the pip command.

In [ ]:
!pip install libtetrabz

## 2, Density of states of free electron
Let us start the simplest case, namely the calculation of density of states of free electron system, $\varepsilon_k = k^2/2$.
$$
D(\varepsilon) = \int d^3 k \delta(\epsilon - \varepsilon_k) = 
4 \pi \int_0^\infty dk k^2 \delta \left(\epsilon - \frac{k^2}{2}\right)
=4 \pi \sqrt{2 \varepsilon}.
$$
As above, we already know the analytical result of $D(\varepsilon)$.
Therefore, it is appropriate to verify the numerical Brillouin-zone integration.

### 2.1, Smearing method
Since the delta function $\delta(\varepsilon)$ has infinity, in the simulation, it is replaced by a smeared function such as the gaussian,
$$
\tilde{\delta}(\varepsilon)=\frac{1}{\sigma \sqrt{\pi}}\exp\left(-\frac{\varepsilon^2}{\sigma^2}\right)
$$


In [ ]:
import numpy

Simulation box of Brillouin zone (must be large enough)

In [ ]:
k_max = 3
e_max = k_max**2 * 0.5

Reciplocal lattice vector

In [ ]:
bvec = 2.0 * numpy.array([[k_max, 0.0, 0.0],
                          [0.0, k_max, 0.0],
                          [0.0, 0.0, k_max]])

$$
\varepsilon_{\bf k} = \frac{k^2}{2}
$$

In [ ]:
ng0 = 10
nb = 1
ng = numpy.array([ng0, ng0, ng0])
eig = numpy.empty([ng[0], ng[1], ng[2], nb], dtype=numpy.float_)
for i0 in range(ng[0]):
    for i1 in range(ng[1]):
        for i2 in range(ng[2]):
            ikvec = numpy.array([i0, i1, i2])
            for ii in range(3):
                if ikvec[ii] * 2 >= ng[ii]:
                    ikvec[ii] += - ng[ii]
            kvec = ikvec[0:3] / ng[0:3]
            kvec[0:3] = kvec.dot(bvec)
            #
            eig[i0, i1, i2, 0] = 0.5 * kvec.dot(kvec)

$$
D(\varepsilon) = \int d^3 k \delta(\varepsilon - \varepsilon_k) \approx
\sum_k \frac{V_{BZ}}{N_k} \tilde{\delta}(\varepsilon - \varepsilon_k)
$$

In [ ]:
vbz = abs(numpy.linalg.det(bvec))

Reasonable choice of broadening width : Average change of enegy between neighboring grid point
$$
\sigma = \left \langle |\nabla_k \varepsilon_k| \right \rangle \Delta k
=  \left \langle \sqrt{2\varepsilon_k} \right \rangle \Delta k
$$

In [ ]:
sigma=numpy.sqrt(2.0*eig).mean()*2.0*k_max/ng0

$$
\tilde{\delta}(\varepsilon)=\frac{1}{\sigma \sqrt{\pi}}\exp\left(-\frac{\varepsilon^2}{\sigma^2}\right)
$$

In [ ]:
e=numpy.linspace(0.0, e_max, 10)
dos=numpy.empty(e.shape[0], dtype=numpy.float_)
for ie in range(e.shape[0]):
    dos[ie] = numpy.exp(-(e[ie]-eig)**2/sigma**2).mean()*vbz/(sigma*numpy.sqrt(numpy.pi))

In [ ]:
import matplotlib.pyplot as plt

Analytical value
$$
D(\varepsilon) = \int d^3 k \delta(\epsilon - \varepsilon_k) = 
4 \pi \int_0^\infty dk k^2 \delta \left(\epsilon - \frac{k^2}{2}\right)
=4 \pi \sqrt{2 \varepsilon}.
$$

In [ ]:
e0=numpy.linspace(0.0, e_max, 100)
dos0=4.0*numpy.pi*numpy.sqrt(2.0*e0)
plt.plot(e0, dos0, label="Exact")
plt.plot(e, dos, label="Smearing", linestyle="None", marker="o")
plt.xlabel("Energy")
plt.ylabel("DOS")
plt.legend()
plt.show()

### 2.2, Tetrahedron method

In [ ]:
import libtetrabz

In [ ]:
wght = libtetrabz.dos(bvec, eig, e)

In [ ]:
dos_t = wght.sum(3).sum(2).sum(1).sum(0)*vbz

In [ ]:
plt.plot(e0, dos0, label="Exact")
plt.plot(e, dos, label="Smearing", linestyle="None", marker="o")
plt.plot(e, dos_t, label="Tetrahedron", linestyle="None", marker="x")
plt.xlabel("Energy")
plt.ylabel("DOS")
plt.legend()
plt.show()

## 3, Lindhard function

Next example is the calculation of polarization function. 
We again consider the simplest case, the free electrons system.
$$
\chi(q) = \int d^3 k \frac{\theta(\varepsilon_{\rm F} - \varepsilon_k) - \theta(\varepsilon_{\rm F} - \varepsilon_{k+q})}{\varepsilon_{k+q} - \varepsilon_k}
$$
If we set $k_{\rm F}=1$,
$$
\chi(q) = 2 \pi \left(1 + \frac{1 - q^2/4}{q} \log\left|\frac{q+2}{q-2} \right|\right)
$$

In [ ]:
k_max = 1.5
bvec = 2.0 * numpy.array([[k_max, 0.0, 0.0],
                         [0.0, k_max, 0.0],
                         [0.0, 0.0, k_max]])
vbz = abs(numpy.linalg.det(bvec))

In [ ]:
ng0 = 10
nb = 1
ng = numpy.array([ng0, ng0, ng0])

In [ ]:
ef = 0.5
qmax = 4.0

In [ ]:
qx = numpy.arange(0.0, qmax, 0.4)

Reasonable choice of broadening width : Average change of enegy between neighboring grid point
$$
\sigma = \left \langle |\nabla_k \varepsilon_k| \right \rangle \Delta k
=  \left \langle \sqrt{2\varepsilon_k} \right \rangle \Delta k
$$

In [ ]:
eig = numpy.empty([ng[0], ng[1], ng[2], nb], dtype=numpy.float_)
for i0 in range(ng[0]):
    for i1 in range(ng[1]):
        for i2 in range(ng[2]):
            ikvec = numpy.array([i0, i1, i2])
            for ii in range(3):
                if ikvec[ii] * 2 >= ng[ii]:
                    ikvec[ii] += - ng[ii]
            kvec = ikvec[0:3] / ng[0:3]
            kvec[0:3] = kvec.dot(bvec)
            eig[i0, i1, i2, 0] = 0.5 * kvec.dot(kvec)
sigma=numpy.sqrt(2.0*eig).mean()*2.0*k_max/ng0  

Broadening method
$$
\chi(q) = \int d^3 k \frac{\tilde{\theta}(\varepsilon_{\rm F} - \varepsilon_k) - \tilde{\theta}(\varepsilon_{\rm F} - \varepsilon_{k+q})}{\varepsilon_{k+q} - \varepsilon_k} \\
\tilde{\theta}(\varepsilon) = \frac{\rm{erfc}(-\varepsilon/\sigma)}{2}=\frac{1}{\sqrt{\pi}}\int_{-\varepsilon}^\infty dt \exp \left[-\left(\frac{t}{\sigma}\right)^2\right] \\
\lim_{\varepsilon_{k+q} \rightarrow \varepsilon_{k}}\frac{\tilde{\theta}(\varepsilon_{\rm F} - \varepsilon_k) - \tilde{\theta}(\varepsilon_{\rm F} - \varepsilon_{k+q})}{\varepsilon_{k+q} - \varepsilon_k} = -\tilde{\delta}(\varepsilon_{\rm F} - \varepsilon_{k+q})
$$

In [ ]:
import math
chi_s = numpy.empty(qx.shape, dtype=numpy.float_)
for iq in range(qx.shape[0]):
    qvec = numpy.array([qx[iq], 0.0, 0.0])
    chi_s[iq] = 0.0
    for i0 in range(ng[0]):
        for i1 in range(ng[1]):
            for i2 in range(ng[2]):
                ikvec = numpy.array([i0, i1, i2])
                for ii in range(3):
                    if ikvec[ii] * 2 >= ng[ii]:
                        ikvec[ii] += - ng[ii]
                kvec = ikvec[0:3] / ng[0:3]
                kvec[0:3] = kvec.dot(bvec)
                eig1 = 0.5 * kvec.dot(kvec) - ef
                
                kvec[0:3] += qvec[0:3]
                eig2 = 0.5 * kvec.dot(kvec) - ef
                
                if abs(eig1 - eig2) < 1.0e-8:
                    chi_s[iq] += numpy.exp(-eig1**2/sigma**2)/(sigma*numpy.sqrt(numpy.pi))
                else:
                    chi_s[iq] += 0.5*(math.erfc(eig1/sigma) - math.erfc(eig2/sigma))/(eig2 - eig1)
    chi_s[iq] *= vbz / ng.prod(0)

Tetrahedron method
$$
\chi(q) = \sum_{\bf k} w_{\bf k}(q) \\
\chi(0) = D(0)
$$

In [ ]:
eig1 = numpy.empty([ng[0], ng[1], ng[2], nb], dtype=numpy.float_)
eig2 = numpy.empty([ng[0], ng[1], ng[2], nb], dtype=numpy.float_)
chi_t = numpy.empty(qx.shape, dtype=numpy.float_)
for iq in range(qx.shape[0]):
    qvec = numpy.array([qx[iq], 0.0, 0.0])
    for i0 in range(ng[0]):
        for i1 in range(ng[1]):
            for i2 in range(ng[2]):
                ikvec = numpy.array([i0, i1, i2])
                for ii in range(3):
                    if ikvec[ii] * 2 >= ng[ii]:
                        ikvec[ii] += - ng[ii]
                kvec = ikvec[0:3] / ng[0:3]
                kvec[0:3] = kvec.dot(bvec)
                eig1[i0, i1, i2, 0] = 0.5 * kvec.dot(kvec) - ef
                
                kvec[0:3] += qvec[0:3]
                eig2[i0, i1, i2, 0] = 0.5 * kvec.dot(kvec) - ef
    if iq == 0:
        e0 = numpy.array([0.0])
        wght = libtetrabz.dos(bvec, eig1, e0)
        chi_t[iq] = wght.sum() * vbz
    else:
        wght = libtetrabz.polstat(bvec, eig1, eig2)
        chi_t[iq] = wght.sum() * vbz * 2.0

$$
\chi(q) = 2 \pi \left(1 + \frac{1 - q^2/4}{q} \log\left|\frac{q+2}{q-2} \right|\right)
$$

In [ ]:
qx0 = numpy.arange(0.0, qmax, 0.011)
chi0 = qx0.copy()
for iq in range(qx0.shape[0]):
    if qx0[iq] < 1.0e-8:
        chi0[iq] = 4.0 * numpy.pi
    elif abs(qx0[iq] - 2.0) < 1.0e-8:
        chi0[iq] = 2.0 * numpy.pi
    else:
        chi0[iq] = 2.0 * numpy.pi*(1.0 + (1.0 - 0.25 * qx0[iq]**2) / qx0[iq] * numpy.log(numpy.abs((qx0[iq] + 2) / (qx0[iq] - 2))))

In [ ]:
plt.plot(qx0, chi0, label="Exact")
plt.plot(qx, chi_s, label="Smearing", linestyle="None", marker="o")
plt.plot(qx, chi_t, label="Tetrahedron", linestyle="None", marker="x")
plt.legend()
plt.xlim(0)
plt.ylim(0)
plt.xlabel("$q / k_F$")
plt.ylabel("$\chi(q)$")
plt.show()

## 4, Multiband, low dimension, and partial DOS
We caonsider the simple honeycomb lattice which has two orbitals (bands) in a unit cell.


In [ ]:
avec = numpy.array([[0.5*numpy.sqrt(3.0), 0.5, 0.0],
                    [0.5*numpy.sqrt(3.0), -0.5, 0.0],
                    [0.0, 0.0, 10.0]])
bvec = 2.0 * numpy.pi*numpy.linalg.inv(avec.transpose())

Set of ${\bf R}=(l,m,n)$ and orbitals pair

Corresponding hopping parameter

In [ ]:
ham_indx = numpy.array([[0,  0, 0, 0, 1],
                        [0,  0, 0, 1, 0],
                        [-1, 0, 0, 0, 1],
                        [1,  0, 0, 1, 0],
                        [0, -1, 0, 0, 1],
                        [0,  1, 0, 1, 0]])
ham = numpy.array([-1.0, -1.0, -1.0, -1.0, -1.0, -1.0])

Diagonalize
$$
H_{i j}({\bf k}) = \sum H_{i j} ({\bf R}) \exp(i{\bf k}\cdot{\bf R})
$$
And compute projection ($|\langle\varphi_{n {\bf k}} |\varphi_{\pm}\rangle|^2$) onto bonding $|\varphi_+\rangle$ and anti-bonding $|\varphi_-\rangle$ orbitals, where
$$
|\varphi_+\rangle = \frac{|\varphi_A\rangle + |\varphi_B\rangle}{\sqrt{2}}\\
|\varphi_-\rangle = \frac{|\varphi_A\rangle - |\varphi_B\rangle}{\sqrt{2}}
$$

In [ ]:
ng0 = 24
nb = 2
ng = numpy.array([ng0, ng0, 1])
eig = numpy.empty([ng[0], ng[1], ng[2], nb], dtype=numpy.float_)
proj = numpy.empty([ng[0], ng[1], ng[2], nb, nb], dtype=numpy.float_)
wfc0 = numpy.array([[1.0, 1.0],
                    [1.0, -1.0]]) / numpy.sqrt(2.0)
for i0 in range(ng[0]):
    for i1 in range(ng[1]):
        for i2 in range(ng[2]):
            kvec = numpy.array([i0, i1, i2]) / ng[0:3]
            ham_k = numpy.zeros([nb, nb], dtype=numpy.complex_)
            for ih in range(ham.shape[0]):
                ham_k[ham_indx[ih, 3], ham_indx[ih, 4]] += ham[ih]*numpy.exp(2.0j*numpy.pi*kvec.dot(ham_indx[ih, 0:3]))
            eig[i0, i1, i2, :], ham_k = numpy.linalg.eigh(ham_k)
            ham_k[:, :] = wfc0.dot(ham_k)
            for ib in range(nb):
                for iwfc in range(2):
                    proj[i0, i1, i2, ib, iwfc] = ham_k[iwfc, ib].real**2 +  ham_k[iwfc, ib].imag**2

In [ ]:
e = numpy.linspace(-3, 3, 100)

In [ ]:
wght = libtetrabz.dos(bvec, eig, e)

In [ ]:
dos = wght.sum(3).sum(2).sum(1).sum(0)
pdos = numpy.zeros([e.shape[0], 2], dtype=numpy.float_)
for ie in range(e.shape[0]):
    for iwfc in range(2):
        pdos[ie, iwfc] = (wght[:,:,:,:,ie]*proj[:,:,:,:,iwfc]).sum()

In [ ]:
plt.plot(e, dos, label="Total")
plt.plot(e, pdos[:, 0], label="Bonding")
plt.plot(e, pdos[:, 1], label="Anti")
plt.xlabel("Energy")
plt.ylabel("DOS")
plt.ylim(0)
plt.legend()
plt.show()

## 5, Simple-cubic tight-binding model, Fermi energy and Fermi surface

In [ ]:
bvec=numpy.array([[10.0,0.0,0.0],
                 [0.0,10.0,0.0],
                 [0.0,0.0,10.0]])

Simple-cubic tight-binding model
$$
\varepsilon_{\bf k} = - \{ \cos(k_x) +\cos(k_y) +\cos(k_z) \}
$$

In [ ]:
ng0 = 10
nb = 1
ng = numpy.array([ng0, ng0, ng0])
eig = numpy.empty([ng[0], ng[1], ng[2], nb], dtype=numpy.float_)
vf = numpy.empty([ng[0], ng[1], ng[2], nb], dtype=numpy.float_)
for i0 in range(ng[0]):
    for i1 in range(ng[1]):
        for i2 in range(ng[2]):
            kvec = 2.0*numpy.pi*numpy.array([i0, i1, i2]) / ng[0:3]
            eig[i0, i1, i2, 0] = - numpy.cos(kvec).sum(0)
            vf[i0, i1, i2, 0] = numpy.sqrt((numpy.sin(kvec)**2).sum(0))

In [ ]:
e=numpy.linspace(-3, 3, 100)

In [ ]:
wght=libtetrabz.dos(bvec,eig,e)

In [ ]:
dos=wght.sum(3).sum(2).sum(1).sum(0)

In [ ]:
plt.plot(e, dos)
plt.xlabel("Energy")
plt.ylabel("DOS")
plt.ylim(0)
plt.show()

In [ ]:
ef, wght, iteration=libtetrabz.fermieng(bvec, eig, 0.5)

In [ ]:
wght.sum(), ef, iteration

In [ ]:
with open("sc.frmsf", "w") as f:
    print(ng[0], ng[1], ng[2], file=f)
    print(1, file=f)
    print(nb, file=f)
    for ii in range(3):
        print(bvec[ii, 0], bvec[ii, 1], bvec[ii, 2], file=f)
    for ib in range(nb):
        for i0 in range(ng[0]):
            for i1 in range(ng[1]):
                for i2 in range(ng[2]):
                    print(eig[i0, i1, i2, ib]-ef, file=f)
    for ib in range(nb):
        for i0 in range(ng[0]):
            for i1 in range(ng[1]):
                for i2 in range(ng[2]):
                    print(vf[i0, i1, i2, ib], file=f)